In [1]:
# Importa as bibliotecas necessárias
import gensim
import gensim.corpora as corpora
import pandas as pd
import re
import spacy

from gensim.models import LdaModel
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
from tqdm import tqdm
tqdm.pandas()

In [14]:
# Carrega o dataset referente às reviews do Airbnb Rio de Janeiro
reviews = pd.read_csv("data/reviews.csv")

In [15]:
# Apresenta as primeiras 5 reviews
reviews.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r\n...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292516 entries, 0 to 292515
Data columns (total 7 columns):
listing_id       292516 non-null int64
id               292516 non-null int64
date             292516 non-null object
reviewer_id      292516 non-null int64
reviewer_name    292516 non-null object
comments         292446 non-null object
language         292076 non-null object
dtypes: int64(3), object(4)
memory usage: 15.6+ MB


In [5]:
# Converte coluna comments para string
reviews["comments"] = reviews["comments"].astype(str)

In [ ]:
# Adiciona coluna com idioma da review
def try_detect_language(text):
    """
    Dado uma string text, tenta obter o idioma predominante.
    
    Argumentos:
    text -- (str) o texto para o qual se deseja detectar o idioma
    Retorna: (str) o código do idioma (e.g. 'en' para inglês). 
             Retorna uma string vazia caso não seja possível detectar o idioma
    """
    try:
        return detect(text)
    except:
        return ""

reviews["language"] = reviews["comments"].apply(try_detect_language)

In [ ]:
# Verifica total de reviews para cada idioma identificado
reviews.language.value_counts().head()

## Preprocessamento dos Dados

In [6]:
nlp_pt = spacy.load("pt")

def preprocess(text, 
               min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE'],
               nlp = nlp_pt): 
    """
    Dado text, min_token_len, e irrelevant_pos, faz o preprocessamento do texto
    
    Argumentos:
    text -- (str) texto a ser pré-processado
    min_token_len -- (int) o menor comprimento de token a ser considerado
    irrelevant_pos -- (list) lista as classes gramaticais a serem ignoradas
    
    Retorna: (str) texto pré processado
    """
    
    # Converte todo os texto para apenas caractéres minúsculos
    text = text.lower()
    
    # Remove termos utilizados pelo airbnb para ocultar informações
    text = re.sub(r'\((.*hidden by airbnb)\)', "", text)
    
    # Processa o texto usando spacy
    doc = nlp(text)
    
    preprocessed = []
    
    # Remove stop words, pontuação, tokens menores que min_token_len e tokens cuja 
    # classe gramatical identificada é irrelavante para análise
    for token in doc:
        if not token.is_stop and not token.is_punct and len(token) >= min_token_len and token.pos_ not in irrelevant_pos:
            # Armazena o lemma do token na variável de saída
            preprocessed.append(token.lemma_)
            
    return " ".join(preprocessed)

def preprocess_pt(text):
    return preprocess(text, nlp = nlp_pt)

In [7]:
reviews_pt = reviews[reviews.language == "pt"]

In [8]:
reviews_pt["clean_comments"] = reviews_pt.comments.progress_apply(preprocess_pt)

100%|██████████| 138315/138315 [1:07:43<00:00, 34.04it/s]    
/Users/marcelle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
reviews_pt.to_csv('data/reviews_pt_preprocessed.csv', index = False)